In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from tqdm import tqdm
from keras.models import Model
from keras.layers import Input, Dense

2024-07-22 01:50:52.212442: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# 데이터 로드
train = pd.read_csv('/Users/jaesolshin/내 드라이브/2024-2/Google ML Bootcamp2024/data/playground1/train.csv')
train = train.sample(frac=0.01, random_state = 42)

In [5]:
# 범주형 변수를 팩터로 변환 (카테고리형)
train.iloc[:,[1,3,4,5,6,7,9]] = train.iloc[:,[1,3,4,5,6,7,9]].astype('category')

# 최소-최대 정규화 (Min-Max 스케일링)
scaler = StandardScaler()
train.iloc[:,[2,8,10]] = scaler.fit_transform(train.iloc[:,[2,8,10]])

# 이분변수 생성: "Annual_Premium" == 2630.0 인 경우
train['Annual_Premium_Binary'] = (train['Annual_Premium'] == 2630.0).astype('category')

# 로그 변환된 "Annual_Premium" 변수 생성
train['Annual_Premium_Log'] = np.where(train['Annual_Premium'] > 0, np.log1p(train['Annual_Premium']), 0)

# 예측에 필요 없는 'id'와 'Annual_Premium' 변수를 드롭
train = train.drop(columns=['id', 'Annual_Premium'])

# 원-핫 인코딩 (One-Hot Encoding)
category_columns = ['Gender', 'Driving_License', 'Region_Code', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Policy_Sales_Channel', 'Annual_Premium_Binary']
train = pd.get_dummies(train, columns=category_columns, drop_first=True, dtype=int)

# XGBoost에서 발생하는 문제 해결
train.columns = train.columns.str.replace('[', '').str.replace(']', '').str.replace('<', '')


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [6]:
# 모델 리스트
models = [
    ('Logistic Regression', LogisticRegression(max_iter=1000, random_state=42)),
    ('Random Forest', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('CatBoost', CatBoostClassifier(random_state=42, verbose=0)),
    ('LightGBM', LGBMClassifier(random_state=42)),
    ('XGBoost', XGBClassifier(random_state=42))
]

# 평가 지표 계산 함수
def evaluate_model(y_true, y_pred, y_proba):
    conf_matrix = confusion_matrix(y_true, y_pred)
    TP = conf_matrix[1, 1]
    FN = conf_matrix[1, 0]
    TN = conf_matrix[0, 0]
    FP = conf_matrix[0, 1]

    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    specificity = TN / (TN + FP) if (TN + FP) != 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    roc_auc = roc_auc_score(y_true, y_proba)

    return {
        'Confusion Matrix': conf_matrix.tolist(),
        'Accuracy': accuracy,
        'Precision': precision,
        'Positive Recall': recall,
        'Specificity': specificity,
        'F1-Score': f1,
        'AUC': roc_auc
    }


In [7]:
# 교차 검증 설정
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
X = train.drop(columns=['Response'])
y = train['Response']

# 결과를 저장할 리스트 초기화
results = []
stacked_predictions = np.zeros((X.shape[0], len(models)))
model_roc_aucs = {}

# 각 모델 학습 및 평가
for model_idx, (name, model) in enumerate(models):
    fold_probas = np.zeros(X.shape[0])
    for train_index, valid_index in tqdm(skf.split(X, y), desc=f"Training {name}"):
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

        # 모델 학습
        model.fit(X_train, y_train)

        # 예측
        valid_y_proba = model.predict_proba(X_valid)[:, 1]  # 양성 클래스의 확률만 저장
        stacked_predictions[valid_index, model_idx] = valid_y_proba
        fold_probas[valid_index] = valid_y_proba

    # 각 모델의 ROC AUC 점수 계산
    roc_auc = roc_auc_score(y, fold_probas)
    model_roc_aucs[name] = roc_auc
    print(f'{name} ROC AUC: {roc_auc}')

Training Logistic Regression: 5it [00:29,  5.93s/it]


Logistic Regression ROC AUC: 0.8508000485942548


Training Random Forest: 5it [05:31, 66.30s/it]


Random Forest ROC AUC: 0.8365779567639171


Training CatBoost: 5it [07:47, 93.40s/it]


...

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 11313, number of negative: 80726
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.110437 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 802
[LightGBM] [Info] Number of data points in the train set: 92039, number of used features: 117
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.122915 -> initscore=-1.965108
[LightGBM] [Info] Start training from score -1.965108


Training LightGBM: 5it [01:11, 14.23s/it]


LightGBM ROC AUC: 0.8648873790457765


Training XGBoost: 5it [02:24, 28.89s/it]


XGBoost ROC AUC: 0.8641579637975969


In [14]:
import joblib
from datetime import datetime
import os

#모델 저장
filepath1 = os.path.join(models_dir, "models.joblib")
filepath2 = os.path.join(models_dir, "stacked_predictions.joblib")

joblib.dump(models, filepath1)
joblib.dump(stacked_predictions, filepath2)

#백업 저장
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

current_dir = os.getcwd()
models_dir = os.path.join(current_dir, "models")
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

filepath1 = os.path.join(models_dir, f"models_{timestamp}.joblib")
filepath2 = os.path.join(models_dir, f"stacked_predictions_{timestamp}.joblib")

joblib.dump(models, filepath1)
joblib.dump(stacked_predictions, filepath2)

['/Users/jaesolshin/Documents/GitHub/kgpg_binary/kfold/models/stacked_predictions_20240722_022105.joblib']

여기서부터 저장된 모델로 진행

In [15]:
import joblib

#모델 경로
filepath1 = os.path.join(models_dir, "models.joblib")
filepath2 = os.path.join(models_dir, "stacked_predictions.joblib")

models = joblib.load(filepath1)
stacked_predictions = joblib.load(filepath2)

In [22]:
results_df = pd.DataFrame(columns=['name', 'roc_auc'])

print("\nModel ROC AUC Scores:")
for model_name, auc_score in model_roc_aucs.items():
    new_result = pd.DataFrame([{'name': model_name, 'roc_auc': auc_score}])
    results_df = pd.concat([results_df, new_result], ignore_index=True)
    print(f"{model_name}: {auc_score}")


Model ROC AUC Scores:
Logistic Regression: 0.8508000485942548
Random Forest: 0.8365779567639171
CatBoost: 0.8661055425228263
LightGBM: 0.8648873790457765
XGBoost: 0.8641579637975969


/var/folders/fx/7qldcfl92nb_g9hqp_lrssxm0000gn/T/ipykernel_60648/2888148930.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_result], ignore_index=True)


In [16]:
# 신경망 메타 모델 정의
input_layer = Input(shape=(len(models),))
dense1 = Dense(10, activation='relu')(input_layer)
output_layer = Dense(1, activation='sigmoid')(dense1)

meta_model = Model(inputs=input_layer, outputs=output_layer)
meta_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])

# 메타 모델 학습
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
meta_model.fit(stacked_predictions, y, epochs=40, batch_size=32, verbose=1, callbacks=[early_stopping])

# StackNN ROC AUC 점수 계산
meta_model_proba = meta_model.predict(stacked_predictions).ravel()
stacknn_roc_auc = roc_auc_score(y, meta_model_proba)
new_result = pd.DataFrame([{'name': 'StackNN', 'roc_auc': stacknn_roc_auc}])
results_df = pd.concat([results_df, new_result], ignore_index=True)

print(results_df)

Epoch 1/50
3596/3596 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - AUC: 0.6898 - loss: 0.4241
Epoch 2/50
3596/3596 ━━━━━━━━━━━━━━━━━━━━ 27s 7ms/step - AUC: 0.8637 - loss: 0.2762
Epoch 3/50
3596/3596 ━━━━━━━━━━━━━━━━━━━━ 28s 4ms/step - AUC: 0.8666 - loss: 0.2711
Epoch 4/50
3596/3596 ━━━━━━━━━━━━━━━━━━━━ 22s 6ms/step - AUC: 0.8641 - loss: 0.2644
Epoch 5/50
3596/3596 ━━━━━━━━━━━━━━━━━━━━ 26s 7ms/step - AUC: 0.8640 - loss: 0.2648
Epoch 6/50
3596/3596 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - AUC: 0.8649 - loss: 0.2644
Epoch 7/50
3596/3596 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - AUC: 0.8668 - loss: 0.2632
Epoch 8/50
3596/3596 ━━━━━━━━━━━━━━━━━━━━ 27s 8ms/step - AUC: 0.8659 - loss: 0.2604
Epoch 9/50
3596/3596 ━━━━━━━━━━━━━━━━━━━━ 27s 7ms/step - AUC: 0.8668 - loss: 0.2608
Epoch 10/50
3596/3596 ━━━━━━━━━━━━━━━━━━━━ 40s 7ms/step - AUC: 0.8670 - loss: 0.2619
Epoch 11/50
3596/3596 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - AUC: 0.8667 - loss: 0.2593
Epoch 12/50
3596/3596 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - AUC: 0.8653 - lo

In [44]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.callbacks import EarlyStopping
from tqdm import tqdm

# 신경망 메타 모델 초기화
meta_model_probas = np.zeros(X.shape[0])
meta_features = np.zeros((X.shape[0], len(models)))

for train_index, valid_index in tqdm(skf.split(meta_features, y), desc="Training Meta Model"):
    X_meta_train, X_meta_valid = stacked_predictions[train_index], stacked_predictions[valid_index]
    y_meta_train, y_meta_valid = y.iloc[train_index], y.iloc[valid_index]

    # 신경망 메타 모델 정의
    input_layer = Input(shape=(len(models),))
    dense1 = Dense(10, activation='relu')(input_layer)
    output_layer = Dense(1, activation='sigmoid')(dense1)

    meta_model = Model(inputs=input_layer, outputs=output_layer)
    meta_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])

    # 조기 종료 콜백 정의
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # 메타 모델 학습
    meta_model.fit(X_meta_train, y_meta_train, epochs=40, batch_size=32, verbose=1, validation_data=(X_meta_valid, y_meta_valid), callbacks=[early_stopping])

    # 메타 모델 예측
    valid_meta_proba = meta_model.predict(X_meta_valid).ravel()
    meta_model_probas[valid_index] = valid_meta_proba

# StackNN ROC AUC 점수 계산
meta_model_proba = meta_model.predict(stacked_predictions).ravel()

# 최종 ROC AUC 점수 계산
stacknn_roc_auc = roc_auc_score(y, meta_model_probas)
new_result = pd.DataFrame([{'name': 'StackNN', 'roc_auc': stacknn_roc_auc}])
results_df = pd.concat([results_df, new_result], ignore_index=True)


Training Meta Model: 0it [00:00, ?it/s]

Epoch 1/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - AUC: 0.6817 - loss: 0.3682 - val_AUC: 0.8648 - val_loss: 0.2675
Epoch 2/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - AUC: 0.8656 - loss: 0.2657 - val_AUC: 0.8641 - val_loss: 0.2649
Epoch 3/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - AUC: 0.8657 - loss: 0.2631 - val_AUC: 0.8653 - val_loss: 0.2630
Epoch 4/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - AUC: 0.8673 - loss: 0.2636 - val_AUC: 0.8654 - val_loss: 0.2623
Epoch 5/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - AUC: 0.8667 - loss: 0.2613 - val_AUC: 0.8651 - val_loss: 0.2618
Epoch 6/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - AUC: 0.8653 - loss: 0.2597 - val_AUC: 0.8651 - val_loss: 0.2616
Epoch 7/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - AUC: 0.8680 - loss: 0.2590 - val_AUC: 0.8653 - val_loss: 0.2613
Epoch 8/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - AUC: 0.8666 - loss: 0.2604 - val_AUC: 0.8651 - val_loss: 0.2611
Epoch 9/40
2877/2877 ━━━━━━━━━━━

Training Meta Model: 1it [10:00, 600.63s/it]

Epoch 1/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - AUC: 0.7367 - loss: 0.3838 - val_AUC: 0.8687 - val_loss: 0.2665
Epoch 2/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - AUC: 0.8645 - loss: 0.2670 - val_AUC: 0.8686 - val_loss: 0.2622
Epoch 3/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - AUC: 0.8660 - loss: 0.2621 - val_AUC: 0.8690 - val_loss: 0.2610
Epoch 4/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - AUC: 0.8647 - loss: 0.2621 - val_AUC: 0.8690 - val_loss: 0.2604
Epoch 5/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - AUC: 0.8662 - loss: 0.2612 - val_AUC: 0.8689 - val_loss: 0.2601
Epoch 6/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - AUC: 0.8638 - loss: 0.2644 - val_AUC: 0.8689 - val_loss: 0.2597
Epoch 7/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - AUC: 0.8639 - loss: 0.2603 - val_AUC: 0.8687 - val_loss: 0.2597
Epoch 8/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - AUC: 0.8663 - loss: 0.2641 - val_AUC: 0.8690 - val_loss: 0.2595
Epoch 9/40
2877/2877 ━━━━━━━━━━━━

Training Meta Model: 2it [15:39, 446.83s/it]

...

Epoch 1/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - AUC: 0.6123 - loss: 0.4049 - val_AUC: 0.8650 - val_loss: 0.2699
Epoch 2/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - AUC: 0.8675 - loss: 0.2663 - val_AUC: 0.8645 - val_loss: 0.2663
Epoch 3/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - AUC: 0.8658 - loss: 0.2646 - val_AUC: 0.8650 - val_loss: 0.2642
Epoch 4/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - AUC: 0.8670 - loss: 0.2642 - val_AUC: 0.8653 - val_loss: 0.2629
Epoch 5/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - AUC: 0.8657 - loss: 0.2619 - val_AUC: 0.8654 - val_loss: 0.2628
Epoch 6/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - AUC: 0.8661 - loss: 0.2604 - val_AUC: 0.8654 - val_loss: 0.2619
Epoch 7/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - AUC: 0.8661 - loss: 0.2615 - val_AUC: 0.8654 - val_loss: 0.2615
Epoch 8/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - AUC: 0.8664 - loss: 0.2614 - val_AUC: 0.8654 - val_loss: 0.2614
Epoch 9/40
2877/2877 ━━━━━━━━━━━

Training Meta Model: 4it [28:24, 406.52s/it]

Epoch 1/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - AUC: 0.6976 - loss: 0.3824 - val_AUC: 0.8646 - val_loss: 0.2676
Epoch 2/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - AUC: 0.8654 - loss: 0.2668 - val_AUC: 0.8642 - val_loss: 0.2637
Epoch 3/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - AUC: 0.8624 - loss: 0.2635 - val_AUC: 0.8658 - val_loss: 0.2625
Epoch 4/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - AUC: 0.8647 - loss: 0.2624 - val_AUC: 0.8656 - val_loss: 0.2620
Epoch 5/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - AUC: 0.8648 - loss: 0.2618 - val_AUC: 0.8655 - val_loss: 0.2617
Epoch 6/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - AUC: 0.8652 - loss: 0.2631 - val_AUC: 0.8657 - val_loss: 0.2614
Epoch 7/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 1811s 630ms/step - AUC: 0.8681 - loss: 0.2572 - val_AUC: 0.8657 - val_loss: 0.2614
Epoch 8/40
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - AUC: 0.8669 - loss: 0.2586 - val_AUC: 0.8656 - val_loss: 0.2612
Epoch 9/40
2877/2877 ━━━━━━━

Training Meta Model: 5it [1:23:22, 1000.44s/it]


3596/3596 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step


In [66]:
print(results_df)

                  name   roc_auc
0  Logistic Regression    0.8508
1        Random Forest  0.836578
2             CatBoost  0.866106
3             LightGBM  0.864887
4              XGBoost  0.864158
5     StackNN(default)  0.866992
6       StackNN(kfold)  0.866928
